# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
weatherpy_data = pd.read_csv('output_data/my_cities.csv')

In [28]:
weatherpy_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bandarbeyla,9.4942,50.8122,81.27,88,46,4.32,SO,1651716440
1,Barrow,71.2906,-156.7887,15.82,79,100,14.97,US,1651716458
2,Albany,42.6001,-73.9662,54.52,92,98,4.52,US,1651716349
3,Ixtapa,20.7000,-105.2000,77.23,78,75,8.05,MX,1651716458
4,Corrente,-3.7367,-43.3394,72.68,98,100,2.75,BR,1651716459


In [29]:
#Drop any rows with null values
print(len(weatherpy_data))
city_weather = weatherpy_data.dropna()
print(len(city_weather))

574
569


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [30]:
# Create a list containing coordinates
coordinates = city_weather[["Lat", "Lng"]].astype(float)
humidity = city_weather["Humidity"].astype(float)

In [33]:
# # Assign the marker layer to a variable
# markers = gmaps.marker_layer(coordinates)
# # Add the layer to the map
# fig.add_layer(markers)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, dissipating=False, max_intensity=100, point_radius = 2)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
#Customize weather conditions
#Temp of 69 - 85
#Winds < 6
#Humidity 40 - 75
#Cloudiness <= 50
ideal_temp = city_weather.loc[(city_weather["Max Temp"]<=85) & (city_weather["Max Temp"]>=69)]
ideal_temp_wind = ideal_temp.loc[(ideal_temp["Wind Speed"]<6)]
ideal_temp_wind_humd = ideal_temp_wind.loc[(ideal_temp_wind["Humidity"]>=40) & (ideal_temp_wind["Humidity"]<=75)]
ideal_temp_wind_humd_clouds = ideal_temp_wind_humd.loc[(ideal_temp_wind_humd["Cloudiness"]<=50)]
ideal_weather = ideal_temp_wind_humd_clouds.reset_index()
ideal_weather = ideal_weather.drop(['index'], axis=1)
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mossamedes,-15.1961,12.1522,78.17,61,1,2.35,AO,1651716460
1,Baharly,38.4362,57.4316,70.11,57,1,2.73,TM,1651716532
2,Saint-Leu,-21.1500,55.2833,74.26,74,10,5.35,RE,1651716570
3,Isla Vista,34.4133,-119.8610,77.99,59,0,1.99,US,1651716572
4,Charters Towers,-20.1000,146.2667,83.03,49,0,3.29,AU,1651716586
5,Lakatoro,-16.0999,167.4164,82.51,75,20,3.76,VU,1651716546
6,Sur,22.5667,59.5289,81.55,53,0,4.09,OM,1651716432
7,Jingdezhen,29.2947,117.2079,77.23,50,0,2.80,CN,1651716601
8,Christchurch,-43.5333,172.6333,71.42,74,23,5.75,NZ,1651716623
9,Kingaroy,-26.5333,151.8333,78.10,50,9,5.23,AU,1651716522


In [35]:
print(len(ideal_weather))

11


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
coordinates = ideal_weather[["Lat", "Lng"]].astype(float)
humidity = ideal_weather["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, dissipating=False, max_intensity=100, point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
hotel_df = ideal_weather.copy()
hotel_df['Hotel Name'] = ""


In [38]:
hotel = []

# set up a parameters dictionary

target_radius = 5000
target_type = "hotel"
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# next start a for loop to iterate thru the rows and get lat & lng for each location
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    # convert response to json
    hotel_results = response.json()
    for i in range(0, len(hotel_results["results"])):
        try:
            if hotel_results["results"][i]["business_status"] == "OPERATIONAL":
                print(f'Results for {city_name}: ')
                print(hotel_results["results"][i]["name"])
                hotel.append(hotel_results["results"][i]["name"])
                break 

        except:
            i += 1 
#     print(f'No local hotel results found.')
#     hotel.append("No results")
        
    print("----------------------------")

    # Print the json (pretty printed) to view keywords needed to pull results
    #print(json.dumps(hotel_results, indent=4, sort_keys=True))
    # results, name
hotel_df = ideal_weather.copy()
hotel_df['Hotel Name'] = hotel


Results for Mossamedes: 
Pavilhão Welwitschia Mirabilis
----------------------------
Results for Baharly: 
TM CELL
----------------------------
Results for Saint-Leu: 
Église du Sacré-Cœur de Saint-Leu
----------------------------
Results for Isla Vista: 
Best Western Plus South Coast Inn
----------------------------
Results for Charters Towers: 
Cattleman's Rest Motor Inn
----------------------------
Results for Lakatoro: 
Vanwods Microfinance Inc.
----------------------------
Results for Sur: 
Sur Plaza Hotel
----------------------------
Results for Jingdezhen: 
Jingdezhen Bureau of Land and Resources
----------------------------
Results for Christchurch: 
YMCA Christchurch
----------------------------
Results for Kingaroy: 
Ascot Lodge Motel Kingaroy
----------------------------
Results for Biu: 
M.I USMAN COMPUTER BUSINESS CENTRE
----------------------------


In [39]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Mossamedes,-15.1961,12.1522,78.17,61,1,2.35,AO,1651716460,Pavilhão Welwitschia Mirabilis
1,Baharly,38.4362,57.4316,70.11,57,1,2.73,TM,1651716532,TM CELL
2,Saint-Leu,-21.1500,55.2833,74.26,74,10,5.35,RE,1651716570,Église du Sacré-Cœur de Saint-Leu
3,Isla Vista,34.4133,-119.8610,77.99,59,0,1.99,US,1651716572,Best Western Plus South Coast Inn
4,Charters Towers,-20.1000,146.2667,83.03,49,0,3.29,AU,1651716586,Cattleman's Rest Motor Inn
5,Lakatoro,-16.0999,167.4164,82.51,75,20,3.76,VU,1651716546,Vanwods Microfinance Inc.
6,Sur,22.5667,59.5289,81.55,53,0,4.09,OM,1651716432,Sur Plaza Hotel
7,Jingdezhen,29.2947,117.2079,77.23,50,0,2.80,CN,1651716601,Jingdezhen Bureau of Land and Resources
8,Christchurch,-43.5333,172.6333,71.42,74,23,5.75,NZ,1651716623,YMCA Christchurch
9,Kingaroy,-26.5333,151.8333,78.10,50,9,5.23,AU,1651716522,Ascot Lodge Motel Kingaroy


In [40]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [41]:
# Add marker layer ontop of heat map
# coordinates = ideal_weather[["Lat", "Lng"]].astype(float)
# humidity = ideal_weather["Humidity"].astype(float)

# fig = gmaps.figure()

markers = gmaps.marker_layer(coordinates, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure

Figure(layout=FigureLayout(height='420px'))